In [1]:
import os 
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tqdm

In [ ]:
# Paramètres 
# Def of the resolution wanted and the domain of interest 
path = ''

resolution_original = 12

lat_bnd = [-0.6, 0.58]
lon_bnd = [-2.6, 0]

Ce fichier extrait les maximas des fichiers sources. à 

# I- Haute Resolution

In [ ]:

def get_max_month_HR (name, year):

    '''
    Input : début du nom des fichier d'un mois une année particulière + année concernée
    Output : dataset avec pour chaque position du domaine d'étude (rlat / rlon) le max de précipitation horaire ainsi que l'année et le mois concerné
    '''

    os.chdir("/work/FAC/FGSE/IDYST/tbeucler/downscaling/HadGEM_driven_COSMO/Future/"+str(year))
    fichiers_month = [fichier for fichier in os.listdir() if fichier.startswith(name)]
    liste_data = []

    for ind in range(len(fichiers_month)) :
        
        data=xr.open_dataset(fichiers_month[ind])
        
        data = data.sel(rlon = slice(*lon_bnd), rlat = slice(*lat_bnd))
        vars_to_keep = ["TOT_PR", 'rlat', 'rlon']  # Liste des variables à conserver
        data = data.drop_vars([var for var in data.variables if var not in vars_to_keep])
        data = data.to_dataframe()
        data= data.reset_index()

        data.rlat = round(data.rlat, 2)
        data.rlon = round(data.rlon, 2)

        liste_data.append(data)

    liste_data = pd.concat(liste_data, ignore_index = True)

    ind = liste_data.groupby(['rlat', 'rlon'])['TOT_PR'].idxmax()
    liste_data = liste_data.loc[ind]

    liste_data['year'] = year
    liste_data['month'] = name[8:10]

    return liste_data


liste_data = []
for year in range(2079, 2090):
    name = 'lffd'+str(year)

    for mois in ['06', '07', '08']:
        print(str(year) + ' : ' + mois)
        data = get_max_month_HR(name + mois, year)
        data.reset_index(inplace = True)
        data.drop(columns = ['index'], inplace = True)
        liste_data.append(data)

liste_data = pd.concat(liste_data, ignore_index = True)

os.chdir(path)
#liste_data.to_csv('data_summer_future_HR.csv', index = False)

# II- Basse Resolution

**Remark :** Les variables '_12' concerne les variables de resolution sans se référer systématiquement à la résolution de 12 km. 

In [ ]:
resolution = int(resolution_original / 2)

# ---------------------------------------------------------------------------

# Les points d'intersections à 2 km
liste_lat_2 = np.arange(lat_bnd[0], lat_bnd[1]+0.01, 0.02)
liste_lon_2 = np.arange(lon_bnd[0], lon_bnd[1]+0.01, 0.02)

# Les points d'intersections à 12 km
liste_lat_12 = liste_lat_2[::resolution]
liste_lon_12 = liste_lon_2[::resolution]

liste_lat_12 = np.append(liste_lat_12, np.max(liste_lat_2) + 0.01)
liste_lon_12 = np.append(liste_lon_12, np.max(liste_lon_2) + 0.01)

# --------------------------------------------------------------------------- 2

# Creation du data dico_coord --> pour chaque localisation à 2 km associer un bloc
lat_indices = np.arange(len(liste_lat_2))
lon_indices = np.arange(len(liste_lon_2))
lat_grid, lon_grid = np.meshgrid(lat_indices, lon_indices)

dico_coord = pd.DataFrame({'rlat': np.array(liste_lat_2)[lat_grid.ravel()], 
                            'rlon': np.array(liste_lon_2)[lon_grid.ravel()]})


dico_coord['block'] = 'X'

ind_block = 0
for lat in tqdm.tqdm(liste_lat_12) :
    for lon in liste_lon_12 :
        liste = (dico_coord.rlon < lon) & (dico_coord.rlat < lat) & (dico_coord.block == 'X')
        if np.sum(liste)>0:
            dico_coord.loc[liste, 'block'] = ind_block
            ind_block += 1


# --------------------------------------------------------------------------- 

# Creation du data data_block --> pour chaque bloc
liste_rlat_12 = dico_coord.groupby('block')['rlat'].mean().values
liste_rlon_12 = dico_coord.groupby('block')['rlon'].mean().values
liste_block = dico_coord.groupby('block')['block'].first().values

data_block = pd.DataFrame({'rlat' : liste_rlat_12, 'rlon' : liste_rlon_12, 'block' : liste_block})


# --------------------------------------------------------------------------- voisin

# Trouver pour chaque position les blocs les plus proches 

def find_min (lat, lon, df):
    
    distance = np.sqrt((df.rlat - lat)**2 + (df.rlon - lon)**2)
    min_distance_index = distance.idxmin()
    
    block = df.loc[min_distance_index, 'block']

    df = df.loc[df['block'] != block]
    
    return block, df


dico_coord['block1'], dico_coord['block2'], dico_coord['block3'], dico_coord['block4'] = 'X', 'X', 'X', 'X'

for ind in tqdm.tqdm(range(len(dico_coord))):
    lon = dico_coord.rlon[ind]
    lat = dico_coord.rlat[ind]
    block = dico_coord.block[ind]

    df = data_block.loc[data_block.block != block].copy()

    block2, df = find_min(lat, lon, df)
    block3, df = find_min(lat, lon, df)
    block4, df = find_min(lat, lon, df)

    dico_coord.loc[(dico_coord.rlat == lat) & (dico_coord.rlon == lon), ['block1', 'block2', 'block3', 'block4']] = [block, block2, block3, block4]
   
dico_coord.rlat = round(dico_coord.rlat, 2)
dico_coord.rlon = round(dico_coord.rlon, 2)

del liste_lat_2, liste_lon_2, liste_lat_12, liste_lon_12, liste_block
del lat_indices, lon_indices, lat_grid, lon_grid, liste

os.chdir(path + "/Topography")
#dico_coord.to_csv('grille2_12.csv', index = False)


In [ ]:
grille = dico_coord[['rlat', 'rlon', 'block']].copy()
grille.rlat = grille.rlat.astype(np.float32)
grille.rlon = grille.rlon.astype(np.float32)

In [ ]:
def get_max_month (name, year):

    os.chdir("/work/FAC/FGSE/IDYST/tbeucler/downscaling/HadGEM_driven_COSMO/Present/"+str(year))
    fichiers_month = [fichier for fichier in os.listdir() if fichier.startswith(name)]
    liste_data = []

    for ind in range(len(fichiers_month)) :

        fichier = fichiers_month[ind]
        data=xr.open_dataset(fichier)
        
        data = data.sel(rlon = slice(*lon_bnd), rlat = slice(*lat_bnd))
        vars_to_keep = ['rlat', 'rlon', 'TOT_PR']  
        data = data.drop_vars([var for var in data.variables if var not in vars_to_keep])
        data = data.to_dataframe()
        data= data.reset_index()

        data.rlat = round(data.rlat, 2)
        data.rlon = round(data.rlon, 2)
        data = pd.merge(data, grille, on = ['rlat', 'rlon'], how = 'left')

        liste_block = data.groupby('block')['block'].first().values
        liste_tot = data.groupby('block')['T0T_PR'].mean().values

        data = pd.DataFrame({'block' : liste_block, 'TOT_PR' : liste_tot})

        liste_data.append(data)

    os.chdir("/work/FAC/FGSE/IDYST/tbeucler/downscaling/louiselargeau")
    liste_data = pd.concat(liste_data, ignore_index = True)
    liste_data = liste_data.reset_index() 

    ind_max = liste_data.groupby('block')['TOT_PR'].idxmax()
    liste_data = liste_data.loc[ind_max].copy()

    liste_data['year'] = year
    liste_data['month'] = name[8:10]

    return liste_data 


liste_data = []
for year in range(1999, 2010):
    name = 'lffd'+str(year)

    for mois in ['06', '07', '08']:
        print(str(year) + ' : ' + mois)
        data = get_max_month(name + mois, year)
        data.reset_index(inplace = True)
        data.drop(columns = ['index'], inplace = True)
        liste_data.append(data)


liste_data = pd.concat(liste_data, ignore_index = True)

os.chdir("/work/FAC/FGSE/IDYST/tbeucler/downscaling/louiselargeau/repos/Downscaling_CM/data/data_low_resolution")
#liste_data.to_csv('data2_summer_present_LR12.csv', index = False)


# III- Topography 

In [ ]:
os.chdir("/work/FAC/FGSE/IDYST/tbeucler/downscaling/HadGEM_driven_COSMO/Topography")
data=xr.open_dataset('lffd19981101000000c.nc')
data = data.sel(rlon = slice(*lon_bnd), rlat = slice(*lat_bnd))
vars_to_keep = ["lat", 'rlat', 'rlon', 'lon', 'HSURF'] 
data = data.drop_vars([var for var in data.variables if var not in vars_to_keep])
data = data.to_dataframe()
data= data.reset_index()
data.drop(columns = ['time'], inplace = True)
data.rename(columns = {'HSURF' : 'alt'}, inplace = True)
data.rlat = data.rlat.astype('float64')
data.rlon = data.rlon.astype('float64')
data.rlat = round(data.rlat, 2)
data.rlon = round(data.rlon, 2)

In [ ]:
# Les points d'intersections à 2 km
liste_lat_2 = np.arange(lat_bnd[0]-0.01, lat_bnd[1]+0.01, 0.02)
liste_lon_2 = np.arange(lon_bnd[0]-0.01, lon_bnd[1]+0.01, 0.02)

# Les points d'intersections block test
lat_limit = liste_lat_2[::30]
lon_limit = liste_lon_2[::24]
lat_limit = liste_lat_2[::54]
lon_limit = liste_lon_2[::30]

lat_limit = np.append(lat_limit, np.max(liste_lat_2) + 0.01)
lon_limit = np.append(lon_limit, np.max(liste_lon_2) + 0.01)

data['block_test'] = 'X'
block_test = 0
for lon in lon_limit :
    for lat in lat_limit :
        liste = (data.rlat < lat) & (data.rlon < lon) & (data.block_test == 'X')
        if np.sum(liste) > 0 :
            data.loc[liste, 'block_test'] = block_test
            block_test +=1

In [ ]:
data['alt_mean5'], data['alt_mean20'] = 'X', 'X'
data['alt_std5'], data['alt_std20'] = 'X', 'X'
data['difff'] = 0

for index, row in tqdm.tqdm(data.iterrows()):
    data.difff = np.sqrt(((data.rlat - row['rlat'])*100)**2 + ((data.rlon - row['rlon'])*100)**2)
    data.loc[index, 'alt_mean5'] = np.mean(data.loc[data.difff < 5, 'alt'])
    data.loc[index, 'alt_std5'] = np.std(data.loc[data.difff < 5, 'alt'])
    data.loc[index, 'alt_mean20'] = np.mean(data.loc[data.difff < 20, 'alt'])
    data.loc[index, 'alt_std20'] = np.std(data.loc[data.difff < 20, 'alt'])

In [ ]:
os.chdir(path)
data.to_csv('grille2.csv', index = False)

# IV- Creation of the data

In this part, we group the information from various data set (High Resolution, Low Resolution, Topography ...) to create the data set needed with all the covariates of interest. 

In [ ]:
nbr = '48' 
os.chdir('/Users/choupie/Documents/ECCE/Private_Downscaling/0_Base/data')
data = pd.merge(pd.read_csv('data2_summer_future_HR.csv'),
                pd.read_csv('Topography/grille2.csv')[['rlat', 'rlon', 'lon', 'lat', 'block_test', 'alt_mean20', 'alt_std20', 'alt']],
                how = 'left', on = ['rlat', 'rlon'])
data.drop(columns = ['time'], inplace = True)
data = pd.merge(data, pd.read_csv('Topography/grille2_'+nbr+'.csv'), on = ['rlat', 'rlon'], how = 'left')

data12 = pd.read_csv('True_Parameters/gev2_param_true_future'+nbr+'.csv')

for i in range(1, 5):
    data120 = data12.copy()
    data120.rename(columns = {'loc' : 'loc'+str(i),
                              'scale' : 'scale' + str(i),
                              'shape' : 'shape'+str(i),
                              'block' : 'block'+str(i)}, inplace = True)
    data = pd.merge(data, data120, on = ['block'+str(i)], how = 'left')

# On ajoute la zone de validation et test
df_liste = []
for test in [0, 5, 6]:
    df = data.loc[data.block_test == test].copy()
    df = df[['rlat', 'rlon', 'block']]
    block_min = min(df.block)
    block_max = max(df.block)

    rlat_min = np.min(df.loc[df.block == block_min, 'rlat']) + (0.12*2) 
    rlat_max = np.max(df.loc[df.block == block_max, 'rlat']) - (0.12*2) 
    df = df.loc[(df.rlat < rlat_min) | (df.rlat > rlat_max)].copy()
    df['Val'] = 1
    df.drop(columns = ['block'], inplace = True)
    df_liste.append(df)

df = pd.concat(df_liste, ignore_index = True)
data = pd.merge(data, df, on = ['rlat', 'rlon'], how = 'left')
data.drop_duplicates(inplace = True)
data.loc[data.Val != 1, 'Val'] = 0
data.loc[(data['block_test'].isin([0, 5, 6])) & (data.Val == 0), 'Test'] = 1
data.loc[data.Test != 1, 'Test'] = 0

# On télécharge
#data.to_csv('data2_12.csv', index = False)
data.to_csv('dataf2_48.csv', index = False)